# Импорта библиотек

In [1]:
import os
import re
import pandas as pd
from typing import Callable

import torch
from llama_cpp import Llama
from huggingface_hub import hf_hub_download
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
from langchain_experimental.utilities.python import PythonREPL

os.environ['HF_HUB_OFFLINE'] = '0'
os.environ['HUGGINGFACE_HUB_CACHE'] = './hugging_face_hub'

/home/zhigul/llm/venv_llm/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Системные инструкции + контекст

In [2]:
### general context about "Napoleon IT Review"
CONTEXT_NAPOLEON_IT = """Что такое "Napoleon IT Отзывы" и зачем нужен это продукт:
Sales & Operations:
-  Отслеживание качества обслуживания: Контроль качества обслуживания клиентов на всех этапах их взаимодействия, от оформления заказа до доставки
-  Анализ удовлетворенности клиентов: Отслеживание метрик удовлетворенности и выявление слабых мест для их устранения
-  Мониторинг конкурентов: Анализ отзывов и оценок конкурентов для понимания их сильных и слабых сторон
-  Оптимизация ассортимента: Выявление наиболее популярных и востребованных позиций, а также тех, которые требуют улучшения
Brand Management & Consumer Market Insights (CMI):
-  Мониторинг восприятия бренда: Изучение отзывов для понимания, как клиенты воспринимают ваш бренд и его уникальные предложения
-  Оптимизация маркетинговых сообщений: Выбор наиболее эффективных сообщений для рекламных кампаний на основе анализа фидбэка
-  Отслеживание реакции на кампании: Мониторинг изменения восприятия до, во время и после рекламных кампаний
RnD:
-  Улучшение качества продуктов: Выявление, какие аспекты вашей продукции клиенты оценивают положительно, а какие требуют улучшения
-  Тестирование нововведений: Анализ отзывов о новых продуктах и услугах, чтобы понять их восприятие и эффективность
-  Анализ трендов: Отслеживание трендов в индустрии, чтобы своевременно внедрять актуальные изменения и оставаться конкурентоспособными
Эта система поможет вам глубже понять потребности и ожидания ваших клиентов, улучшить качество обслуживания и ассортимент продукции, что в конечном итоге повысит лояльность покупателей и увеличит продажи."""

SALES_SCRIPT = """1. Поприветствуй клиента и представься сам.
2. Расскажи о выгодном предложении, которое ты ему принес.
3. Выдели ключевые преимущества твоего предложения и то, как это решение может помочь бизнесу клиента (опиши очень кратко).
4. Подтолкни клиента к ответу, задав вопрос о дальнейшей коммуникации и поинтересуйся, как клиент оценивает предоставленное предложение.
5. Не забудь попращаться на дружественной ноте и зафиксировать, что ждешь ответ от клиента."""


### general system prompts
SYS_PROMPT_INIT_DIALOGUE = lambda context_1, sales_script: f"""Ты - AI-ассистент в роли профессионального агента-продавца, опыт которого в продажах больше 10 лет.
Ты инициализируешь диалог с клиентом через переписку и твоя цель - заинтересовать его продуктом, который ты предлагаешь.
Ты должен быть уважительным.
Не зацикливайся и выдавай убедительные предложения, которые могут заинтересовать клиента.
Твои ответы должны быть только на Русском.

Продукт, который ты предлагаешь описан тут:
{context_1}

В своем ответе используй следующие подсказки:
{sales_script}

Строго соблюдай каждую инструкцию и структуру ответа.
Покажи по-настоящему свои навыки продавца и заинтересуй клиента:
"""

USER_INIT_PROMPT = "Следуй точно инструкциями и начни диалог с клиентом:"


SYS_PROMPT_CONTINUE_DIALOGUE = lambda context_2: f"""Ты - профессиональный агент-продавец, опыт которого в продажах больше 10 лет.
Ты ведешь диалог через переписку с другим бизнесом, которому предлагаешь сотрудничество по продукту "Napoleon IT Отзывы".
"Napoleon IT Отзывы" - платформа, которая предлагает бизнесам следить за актуальной информацией о своих продуктах и услугах.
Таким образом, другие бизнесы могут стать нашими клиентами и получать только релевантную и достовернную информацию по своим продуктам.
Ты должен обрабатывать любые вопросы клиента по его продукту, используя контекст.
Отвечай только на Русском.

Релевантные данные о продукте клиента приведены ниже:
{context_2}

Обработай запрос клиента и выдай точную информацию:
"""

SYS_PROMPT_FOR_INTENTION = """Ты модель-классификатор, которая умеет четко определять намерение пользователя по его запросу.
На вход тебе поступает пользовательский запрос и твоя задача - бинарно определить, относится ли данный запрос к продукту "Napoleon IT Отзывы".
Строго выдай только бинарную метку "да" либо "нет" - без дополнительных комментариев и текста:
"""

SYS_PROMPT_TEXT_2_PANDAS = lambda path_to_data, columns, example: f"""Ты профессиональный программист и ML-разработчик.
Ты пишешь качественный Python-код, используя библиотеку pandas, datetime, numpy и другие в зависимости от задачи.
Также тебе предоставлен датасет в формате .csv по пути {path_to_data}.
Датасет имеет следующие столбцы: {columns}.
Пример строки представлен тут: {example}.

Напиши код на Python, который сможет достать наиболее релевантные данные из датасета по запросу пользователя.
Строго соблюдай следующий формат ответа:

```
[Программный код]
```
Никаких пояснений и комментариев до и после кода писать не нужно, СТРОГО следуй этому правилу:
"""

# Класс SalesAgent

In [3]:
class SalesAgent:

    def __init__(self, 
                 model_id, 
                 repo_gguf_id, 
                 filename_gguf, 
                 load_gguf_config: dict) -> None:

        self.model_id = model_id
        self.repo_gguf_id = repo_gguf_id
        self.filename_gguf = filename_gguf
        self.load_gguf_config = load_gguf_config

        self.model = self.load_gguf_model()


    def load_gguf_model(self) -> Llama:

        model_path = hf_hub_download(repo_id=self.repo_gguf_id, filename=self.filename_gguf)
        
        model = Llama(
            model_path,
            **self.load_gguf_config
        )
        return model
    

    def inference_agent(self, messages: list[dict]) -> str:
        response = self.model.create_chat_completion(messages)

        return response['choices'][0]['content']
    

    def formating_chat_template(self, 
                                system: Callable,
                                context: list[str],
                                user: list[str],
                                assistan: list[str]) -> list[dict]:
        messages = [
                {'role': 'system', 'content': system(*context)}
        ]

        for assistant_replica, user_replica in zip(assistan, user):
            
            messages.append(assistant_replica)
            messages.append(user_replica)

        return messages

In [4]:
model_id = 'microsoft/Phi-3.5-mini-instruct'
repo_gguf_id = 'QuantFactory/Phi-3.5-mini-instruct-GGUF'
filename_gguf = 'Phi-3.5-mini-instruct.Q8_0.gguf'

LOAD_GGUF_CONFIG = {
    'n_gpu_layers': 0, # not GPUs
    'n_threads': 32, # for CPUs
    'n_ctx': 4096  # length context ...
}

model = SalesAgent(model_id,
                   repo_gguf_id,
                   filename_gguf,
                   LOAD_GGUF_CONFIG)

llama_model_loader: loaded meta data with 36 key-value pairs and 197 tensors from /home/zhigul/.cache/huggingface/hub/models--QuantFactory--Phi-3.5-mini-instruct-GGUF/snapshots/cb88a6c948766fc240bc811bf4d7f2f4ba94afaf/Phi-3.5-mini-instruct.Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = phi3
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Phi 3.5 Mini Instruct
llama_model_loader: - kv   3:                           general.finetune str              = instruct
llama_model_loader: - kv   4:                           general.basename str              = Phi-3.5
llama_model_loader: - kv   5:                         general.size_label str              = mini
llama_model_loader

# Конвейер Text2Pandas

In [5]:
USER, ASSISTANT = [], [] # для сохранения контекста диалога
PATH_TO_DATASET = 'dataset/cleaned_coffee.csv'

delimeter = '\n\n' + ('='* 100) + '\n\n'

# Interpreter Python
REPL = PythonREPL()

cleaning_t2pandas = lambda answer: re.sub('python', '', answer.split('```')[1]).strip()

### загрузка датасета
dataset = pd.read_csv(PATH_TO_DATASET)
dataset.head()

FileNotFoundError: [Errno 2] No such file or directory: 'dataset/cleaned_coffee.csv'

In [18]:
### Инициализация диалога
messages_init = [
    {'role': 'system', 'content': SYS_PROMPT_INIT_DIALOGUE(CONTEXT_NAPOLEON_IT, SALES_SCRIPT)},
    {'role': 'user', 'content': USER_INIT_PROMPT}
]

started_message_to_customer = model.inference_agent(messages_init)
ASSISTANT.append(started_message_to_customer) # запоминаем диалог со стороны ассистента

print(f'STEP #1:\n\nСООБЩЕНИЕ МОДЕЛИ ДЛЯ ИНИЦИАЛИЗАЦИИ: {started_message_to_customer}', end=delimeter)


### Продолжение диалога в формате ожидания ответа от клиента
while True:

    # приходит ответ от клиента
    customer = input()
    USER.append(customer) # запоминаем диалог со стороны клиента

    print(f'STEP #2:\n\nОТВЕТ КЛИЕНТА: {customer}', end=delimeter)

    # проверяем intent
    messages_intent = [
        {'role': 'system', 'content': SYS_PROMPT_FOR_INTENTION},
        {'role': 'user', 'content': customer}
    ]
    label_intention = model.inference_agent(messages_intent)

    print(f'ВЫЯВЛЯЕМ НАМЕРЕНИЕ:\n\nНАМЕРЕНИЕ: {label_intention}', end=delimeter)

    # без RAG'a
    if 'да' in label_intention:

        print('КОНТЕКСТ:\n{CONTEXT_NAPOLEON_IT}', end=delimeter)

        messages = model.formating_chat_template(system=SYS_PROMPT_CONTINUE_DIALOGUE,
                                                 context=CONTEXT_NAPOLEON_IT,
                                                 user=USER,
                                                 assistant=ASSISTANT)
        agent = model.inference_agent(messages)

    # с RAG'ом
    elif 'нет' in label_intention:

        # RAG
        path_to_data, columns, example = PATH_TO_DATASET, dataset.columns.to_list(), dataset.sample(1)

        messages = model.formating_chat_template(system=SYS_PROMPT_TEXT_2_PANDAS,
                                                 context=[path_to_data, columns, example],
                                                 user=USER,
                                                 assistant=ASSISTANT)
        rag_agent = model.inference_agent(messages)

        print(f'TEXT_2_PANDAS ЗАПРОС МОДЕЛИ:\n{rag_agent}', end=delimeter)

        CONTEXT_RAG = REPL.run(rag_agent)
        CONTEXT_RAG = cleaning_t2pandas(CONTEXT_RAG)

        print(f'ПОЛУЧЕННЫЙ КОНТЕКСТ С RAGA:\n{CONTEXT_RAG}', end=delimeter)
        
        messages = model.formating_chat_template(system=SYS_PROMPT_CONTINUE_DIALOGUE,
                                                 context=[CONTEXT_RAG],
                                                 user=customer,
                                                 assistant=ASSISTANT)
        agent = model.inference_agent(messages)

        print(f'ОТВЕТ МОДЕЛИ ПО RAGy:\n{agent}', end=delimeter)


    ASSISTANT.append(agent) # запоминаем диалог со стороны ассистента

Llama.generate: 1032 prefix-match hit, remaining 1 prompt tokens to eval
